# StellarGraph examples

The StellarGraph object is a subclass of NetworkX MultiGraph, and has all the features built into NetworkX.


Some example graphs

In [265]:
import networkx as nx
from stellar.data.stellargraph import *
from stellar.data.loader import *

In [13]:
%reload_ext autoreload
%autoreload 2

We can create StellarGraph objects from NetworkX graphs:

By default the edge types will be empty strings.

In [243]:
G = StellarGraph(nx.karate_club_graph())
G

StellarGraph: Undirected multigraph
    Nodes: 34, Edges: 78

Note that certain NX operations will return a NetworkX object, not a StellarGraph object:

In [223]:
nx.subgraph(G, [0])

In [224]:
G.copy()

In [345]:
G.to_directed()

To create a heterogenous graph we need to specify node and/or edge labels:

In [341]:
G = StellarGraph()
G.add_nodes_from([0, 1, 2, 3], label="movie")
G.add_nodes_from([4, 5], label="user")
e_keys = G.add_edges_from([(0, 4), (1, 4), (1, 5), (2, 4), (3, 5)], label="rating")
#e_keys = G.add_edges_from([(4, 5)], label="friend")

We can query the object to find more information about the Graph types

In [342]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 6, Edges: 5

 Node types:
  movie: [4]
    Edge types: movie-rating->user
  user: [2]
    Edge types: user-rating->movie

 Edge types:
    movie-rating->user: [5]



Add some more edges to form a real multigraph

In [343]:
e_keys = G.add_edges_from([(0, 4), (1, 4)], label="another")

In [344]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 6, Edges: 7

 Node types:
  movie: [4]
    Edge types: movie-another->user, movie-rating->user
  user: [2]
    Edge types: user-another->movie, user-rating->movie

 Edge types:
    movie-another->user: [2]
    movie-rating->user: [5]



## Graph Schema

The graph schema gives information on the node and edge types 

In [321]:
gs = G.create_graph_schema(create_type_maps=True)
gs

GraphSchema:
node type: movie
   movie -- rating -> user
node type: user
   user -- rating -> movie

The node edges and types are available. The ordering of these is important and maintained in all methods:

In [324]:
gs.schema['user']

[EdgeType(n1='user', rel='rating', n2='movie')]

In [322]:
gs.node_types

['movie', 'user']

In [281]:
gs.edge_types

[EdgeType(n1='movie', rel='another', n2='user'),
 EdgeType(n1='movie', rel='rating', n2='user'),
 EdgeType(n1='user', rel='another', n2='movie'),
 EdgeType(n1='user', rel='friend', n2='user'),
 EdgeType(n1='user', rel='rating', n2='movie')]

Nodes and edges have indices that should be extracted using the node_index and edge_index functions:

In [286]:
gs.node_index('movie')

0

In [282]:
gs.edge_index(('movie', 'rating', 'user'))

1

The graph schema can be used to quickly look up node and edge types:

In [272]:
gs.get_node_type(0)

'movie'

We can easily do filtering of nodes and edges based on types:

The edge type is specified to be a triple:

In [283]:
[
    n for n in G if gs.get_node_type(n) == 'user'
]

[4, 5]

In [273]:
gs.get_edge_type((0,4,0))

EdgeType(n1='movie', rel='rating', n2='user')

In [284]:
[
    e for e in G.edges(keys=True) if gs.get_edge_type(e) == ('movie', 'rating', 'user')
]

[(0, 4, 0), (1, 4, 0), (1, 5, 0), (2, 4, 0), (3, 5, 0)]

This is a named tuple, so we can compare edges with tuples:

In [274]:
gs.get_edge_type((0,4,0)) == ('movie', 'rating', 'user')

True

In [276]:
gs.edge_index(('movie', 'rating', 'user'))

1

In [278]:
[
    n for n in G if gs.get_node_type(n) == 'user'
]

[4, 5]

Note that StellarGraph assumes there is only a single edge with the same label between two noddes.

This isn't enforced ATM so this will break things:

In [221]:
[
    e for e in G.edges(keys=True) if gs.get_edge_type(e) == ('movie', 'another', 'user')
]

[(0, 4, 1),
 (0, 4, 2),
 (0, 4, 3),
 (0, 4, 4),
 (0, 4, 5),
 (1, 4, 1),
 (1, 4, 2),
 (1, 4, 3),
 (1, 4, 4),
 (1, 4, 5)]

In [237]:
gs.edge_types_for_node_type('user')

[EdgeType(n1='user', rel='another', n2='movie'),
 EdgeType(n1='user', rel='friend', n2='user'),
 EdgeType(n1='user', rel='rating', n2='movie')]

For sub-graph sampling we can obtain the tree of node and edge types from a head node

In [240]:
gs.get_sampling_tree(['user'], n_hops=2)

[('0',
  'user',
  [('0#0',
    EdgeType(n1='user', rel='another', n2='movie'),
    [('0#0_0', EdgeType(n1='movie', rel='another', n2='user'), []),
     ('0#0_1', EdgeType(n1='movie', rel='rating', n2='user'), [])]),
   ('0#1',
    EdgeType(n1='user', rel='friend', n2='user'),
    [('0#1_0', EdgeType(n1='user', rel='another', n2='movie'), []),
     ('0#1_1', EdgeType(n1='user', rel='friend', n2='user'), []),
     ('0#1_2', EdgeType(n1='user', rel='rating', n2='movie'), [])]),
   ('0#2',
    EdgeType(n1='user', rel='rating', n2='movie'),
    [('0#2_0', EdgeType(n1='movie', rel='another', n2='user'), []),
     ('0#2_1', EdgeType(n1='movie', rel='rating', n2='user'), [])])])]

Or the sampling adjacency list schema (for GraphSAGE)

In [350]:
gs.schema['user']

[EdgeType(n1='user', rel='rating', n2='movie')]

In [346]:
gs.get_type_adjacency_list(['user', 'movie'], n_hops=2)

[('user', [2]),
 ('movie', [3]),
 ('movie', [4]),
 ('user', [5]),
 ('user', []),
 ('movie', [])]

## Loading graphs from different formats

EPGM formats are ideal for storing heterogeneous graphs. They are handled through a helper function:

In [288]:
graph_loc = "../tests/resources/data/yelp/yelp.epgm"
G = from_epgm(graph_loc)

...reading ../tests/resources/data/yelp/yelp.epgm/graphs.json using utf-8 encoding...
...reading ../tests/resources/data/yelp/yelp.epgm/vertices.json using utf-8 encoding...
...reading ../tests/resources/data/yelp/yelp.epgm/edges.json using utf-8 encoding...
Converting the EPGM graph 59be1422c4785d276d9c3658 to NetworkX graph...
Graph statistics: 621 nodes, 6832 edges


In [289]:
print(G.info())

StellarGraph: Undirected multigraph
    Nodes: 621, Edges: 6832

  Node types:
    business: [10]
        Attributes: {'BusinessAcceptsCreditCards', 'BikeParking', 'state', 'GoodForKids', 'Smoking', 'city', 'name', 'categories', 'NoiseLevel', 'DriveThru', 'WiFi', 'DogsAllowed', 'yelpId', 'Alcohol', 'RestaurantsPriceRange2', 'stars', 'HappyHour', 'HasTV'}
    Edge types: business-locatedIn->city, business-towards->review
    city: [3]
        Attributes: {'name'}
    Edge types: city-locatedIn->business, city-state->state
    review: [37]
        Attributes: {'useful', 'userId', 'businessId', 'yelpId', 'stars'}
    Edge types: review-towards->business, review-writes->user
    state: [2]
        Attributes: {'name'}
    Edge types: state-state->city
    user: [569]
        Attributes: {'elite', 'useful', 'compliment_more', 'cool', 'compliment_profile', 'compliment_note', 'compliment_funny', 'compliment_cute', 'compliment_hot', 'name', 'compliment_writer', 'yelpId', 'compliment_cool', 'fu

In [307]:
G.create_graph_schema()

GraphSchema:
node type: business
   business -- locatedIn -> city
   business -- towards -> review
node type: city
   city -- locatedIn -> business
   city -- state -> state
node type: review
   review -- towards -> business
   review -- writes -> user
node type: state
   state -- state -> city
node type: user
   user -- friend -> user
   user -- writes -> review